In [ ]:
%pip install -qU langchain-openai

In [ ]:
%pip install langchain

In [ ]:
import langchain

# LLM

In [ ]:
from langchain_openai import OpenAI

In [ ]:
llm = OpenAI()

In [ ]:
llm('A cat climbed the wall,')

In [ ]:
llm.invoke('A cat climbed the wall,')

In [ ]:
result = llm.generate(['A cat climbed the wall,', 'Jack and Jill'])

In [ ]:
result.schema()

In [ ]:
result.llm_output

In [ ]:
result.generations

In [ ]:
print(result.generations[0][0].text)

# Chat Models

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
chat = ChatOpenAI()

In [ ]:
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [ ]:
result = chat([SystemMessage(content='You are a Bank employee'), HumanMessage(content='Tell me a joke about your life')])

In [ ]:
print(result.content)

In [ ]:
result = chat.generate([
    [SystemMessage(content='You are a lazy Bank employee, who hates your job'),
     HumanMessage(content='Tell me a joke about your life')],
    [SystemMessage(content='You are a very Professional Bank employee, who loves your job'),
     HumanMessage(content='Tell me a joke about your life')]
    ])

In [ ]:
result.llm_output

In [ ]:
result.generations

In [ ]:
print(result.generations[0][0].text)

In [ ]:
print(result.generations[1][0].text)

In [ ]:
result = chat(
    [SystemMessage(content='You are a lazy Bank employee, who hates your job'),
     HumanMessage(content='Tell me a joke about your bank life')],
     temperature=0, max_tokens=20
    )

In [ ]:
print(result.content)

In [ ]:
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()

In [ ]:
llm.predict('Tell me a fact about Mars')

In [ ]:
llm.predict('Tell me a fact about Mars')

# Prompt Templates

In [ ]:
from langchain_core.prompts import PromptTemplate

In [ ]:
no_input_prompt = PromptTemplate(input_variables=[],
                                template = 'Tell me a fact')

In [ ]:
print(llm(no_input_prompt.format()))

In [ ]:
single_input_prompt = PromptTemplate(input_variables=['topic'],
                                    template='Tell me a fact about {topic}')

In [ ]:
single_input_prompt.format(topic= 'Pluto')

In [ ]:
print(llm(single_input_prompt.format(topic= 'Pluto')))

# Chat Prompt Template

In [ ]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

In [ ]:
system_template = "You are an AI recipe Assistant that specializes in {dietary_preference} dishes that can be prepared in {cooking_time}"

In [ ]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [ ]:
human_template = "{recipe_request}"

In [ ]:
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
system_message_prompt.input_variables

In [ ]:
human_message_prompt.input_variables

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [ ]:
chat_prompt.input_variables

In [ ]:
prompt = chat_prompt.format_prompt(cooking_time='60 min', dietary_preference='Vegan', recipe_request='Quick Snack').to_messages()

In [ ]:
result = chat(prompt)

In [ ]:
print(result.content)

# Few Shot Prompts

In [ ]:
system_template = "You are an AI recipe Assistant that specializes in giving the wacky definitions for acronyms"

In [ ]:
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [ ]:
# few shot examples
acronyms = "A.I.R"
example_input_one = HumanMessagePromptTemplate.from_template(acronyms)

wacky_abbrevations = "Absolute Insane Rabbit"
example_output_one = HumanMessagePromptTemplate.from_template(wacky_abbrevations)

In [ ]:
human_template = "{new_acronym}"

In [ ]:
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
Chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, example_input_one, example_output_one, human_message_prompt])

In [ ]:
Chat_prompt.input_variables

In [ ]:
request = Chat_prompt.format_prompt(new_acronym="P.E.T").to_messages()

In [ ]:
request

In [ ]:
few_shot_result = chat(request)

In [ ]:
print(few_shot_result.content)

# Parsing outputs

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [ ]:
parser = CommaSeparatedListOutputParser()

In [ ]:
parser.get_format_instructions()

In [ ]:
reply = "red, blue, green"

In [ ]:
parser.parse(reply)

In [ ]:
human_template = '{request}\n{format_instructions}'
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
human_message_prompt.input_variables

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

In [ ]:
request = chat_prompt.format_prompt(request='Give me five breeds of dogs', format_instructions= parser.get_format_instructions()).to_messages()

In [ ]:
request

In [ ]:
response = chat(request)

In [ ]:
response

In [ ]:
print(response.content)

In [ ]:
print(parser.parse(response.content))

# Document Loaders

In [ ]:
from langchain_community.document_loaders import HNLoader

In [ ]:
loader = HNLoader("https://news.ycombinator.com/item?id=34817881")

In [ ]:
data = loader.load()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
chat = ChatOpenAI()

In [ ]:
print(data[0].page_content)

In [ ]:
human_template = "Give me a short summary on the following Hackernews Community comment: \n{comment}"

In [ ]:
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([human_message_prompt])

In [ ]:
chat_prompt.input_variables

In [ ]:
prompt = chat_prompt.format_prompt(comment=data[0].page_content).to_messages()

In [ ]:
response = chat(prompt)

In [ ]:
print(response.content)

# Text Embedding

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings()

In [ ]:
text = 'HuggingFace is the best AI compnay'

In [ ]:
text_query_result = embeddings.embed_query(text)

In [ ]:
len(text_query_result)

In [ ]:
text_query_result

# Document Embeddings

In [6]:
loader = HNLoader("https://news.ycombinator.com/item?id=34817881")

In [7]:
data = loader.load()

In [8]:
data

[Document(page_content="delta_p_delta_x on Feb 17, 2023  \n             | next [–] \n\nAstrophysical and cosmological simulations are often insightful. They're also very cross-disciplinary; besides the obvious astrophysics, there's networking and sysadmin, parallel computing and algorithm theory (so that the simulation programs are actually fast but still accurate), systems design, and even a bit of graphic design for the visualisations.Some of my favourite simulation projects:- IllustrisTNG: https://www.tng-project.org/- SWIFT: https://swift.dur.ac.uk/- CO5BOLD: https://www.astro.uu.se/~bf/co5bold_main.html (which produced these animations of a red-giant star: https://www.astro.uu.se/~bf/movie/AGBmovie.html)- AbacusSummit: https://abacussummit.readthedocs.io/en/latest/And I can add the simulations in the article, too.", metadata={'source': 'https://news.ycombinator.com/item?id=34817881', 'title': 'What Lights the Universe’s Standard Candles?'}),
 Document(page_content="froeb on Feb 18

In [10]:
type(data), len(data)

(list, 6)

In [ ]:
query_result = embeddings.embed_documents([document.page_content for document in data])

In [ ]:
len(query_result)

In [ ]:
len(query_result[0])

In [ ]:
query_result

In [2]:
from langchain.document_loaders import TextLoader

ModuleNotFoundError: No module named 'langchain'

In [16]:
import os

os.getcwd()

'c:\\Users\\hemachandran.d\\OneDrive - Tredence\\Documents\\Github\\LangChain_bootcamp\\cookbook'

In [17]:
loader = loader = TextLoader(os.getcwd()+"\yelp_review.txt")
data = loader.load()

In [18]:
data

[Document(page_content="TITLE: AN ABSOLUTE DELIGHT! A CULINARY HAVEN!\n\nREVIEW:\nOH MY GOODNESS, WHERE DO I BEGIN? THIS RESTAURANT IS ABSOLUTELY PHENOMENAL! I WENT THERE LAST NIGHT WITH MY FRIENDS, AND WE WERE BLOWN AWAY BY THE EXPERIENCE!\n\nFIRST OF ALL, THE AMBIANCE IS OUT OF THIS WORLD! THE MOMENT YOU STEP INSIDE, YOU'RE GREETED WITH A WARM AND INVITING ATMOSPHERE. THE DECOR IS STUNNING, AND IT IMMEDIATELY SETS THE TONE FOR AN UNFORGETTABLE DINING EXPERIENCE.\n\nNOW, LET'S TALK ABOUT THE FOOD! WOW, JUST WOW! THE MENU IS A PARADISE FOR FOOD LOVERS. EVERY DISH WE ORDERED WAS A MASTERPIECE. THE FLAVORS WERE BOLD, VIBRANT, AND EXPLODED IN OUR MOUTHS. FROM STARTERS TO DESSERTS, EVERY BITE WAS PURE BLISS!\n\nTHEIR SEAFOOD PLATTER IS A MUST-TRY! THE FRESHNESS OF THE SEAFOOD IS UNMATCHED, AND THE PRESENTATION IS SIMPLY STUNNING. I HAVE NEVER TASTED SUCH DELICIOUS AND PERFECTLY COOKED SEAFOOD IN MY LIFE. IT'S A SEAFOOD LOVER'S DREAM COME TRUE!\n\nTHE SERVICE WAS EXEMPLARY. THE STAFF WAS AT

In [19]:
type(data), len(data)

(list, 1)

# Vector Store

In [ ]:
%pip install langchain-chroma

In [26]:
import chromadb
from langchain_community.embeddings import HuggingFaceEmbeddings

TextSplitter

In [20]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
pip install pypdf

In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
#LOAD DCUMENT ---> SPLIT CHUNKS

#EMBEDDINGS ---> EMBEDDING CHUNKS --> VECTORS

#VECTOR CHUNKS ---> SAVE CHROMADB

#QUERY VECTOR STORE ---> SIMILARITY SEARCH

In [ ]:
loader = PyPDFLoader('/teamspace/studios/this_studio/Databricks-Big-Book-Of-GenAI-FINAL.pdf')
data = loader.load()

In [ ]:
data

In [21]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(data)

In [23]:
docs

[Document(page_content="TITLE: AN ABSOLUTE DELIGHT! A CULINARY HAVEN!\n\nREVIEW:\nOH MY GOODNESS, WHERE DO I BEGIN? THIS RESTAURANT IS ABSOLUTELY PHENOMENAL! I WENT THERE LAST NIGHT WITH MY FRIENDS, AND WE WERE BLOWN AWAY BY THE EXPERIENCE!\n\nFIRST OF ALL, THE AMBIANCE IS OUT OF THIS WORLD! THE MOMENT YOU STEP INSIDE, YOU'RE GREETED WITH A WARM AND INVITING ATMOSPHERE. THE DECOR IS STUNNING, AND IT IMMEDIATELY SETS THE TONE FOR AN UNFORGETTABLE DINING EXPERIENCE.\n\nNOW, LET'S TALK ABOUT THE FOOD! WOW, JUST WOW! THE MENU IS A PARADISE FOR FOOD LOVERS. EVERY DISH WE ORDERED WAS A MASTERPIECE. THE FLAVORS WERE BOLD, VIBRANT, AND EXPLODED IN OUR MOUTHS. FROM STARTERS TO DESSERTS, EVERY BITE WAS PURE BLISS!\n\nTHEIR SEAFOOD PLATTER IS A MUST-TRY! THE FRESHNESS OF THE SEAFOOD IS UNMATCHED, AND THE PRESENTATION IS SIMPLY STUNNING. I HAVE NEVER TASTED SUCH DELICIOUS AND PERFECTLY COOKED SEAFOOD IN MY LIFE. IT'S A SEAFOOD LOVER'S DREAM COME TRUE!\n\nTHE SERVICE WAS EXEMPLARY. THE STAFF WAS AT

In [22]:
type(docs), len(docs)

(list, 2)

In [1]:
embeddings = HuggingFaceEmbeddings()

NameError: name 'HuggingFaceEmbeddings' is not defined

In [ ]:
result = embeddings.embed_documents([doc.page_content for doc in docs])
result

In [ ]:
type(result), len(result),

In [ ]:
db = Chroma.from_documents(docs, embeddings, persist_directory= "./genai_guide")

In [ ]:
db.persist()

In [ ]:
similar_docs = db.similarity_search("When was the DBRX released?")

In [ ]:
print(similar_docs[0].page_content)

# Retriever

In [ ]:
retriever = db.as_retriever()

In [ ]:
results = retriever.get_relevant_documents("How the DBRX is Evaluated?")

In [ ]:
print(results[3].page_content)

# MultiQuery Retriever

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever

In [ ]:
%pip install wikipedia

In [ ]:
from langchain.document_loaders import WikipediaLoader

In [ ]:
loader = WikipediaLoader(query='MKUltra')
data = loader.load()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
textsplitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = textsplitter.split_documents(data)

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings()

In [ ]:
db = Chroma.from_documents(docs, embeddings, persist_directory= "./mkultra")
db.persist()

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI()

In [ ]:
retriever_from_llm = MultiQueryRetriever.from_llm(retriever = db.as_retriever(), llm = llm)

In [ ]:
question= "When was this declassified"

In [ ]:
import logging
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [ ]:
# WILL NOT ANSWER QUERY DIRECTLY
# SEE MULTIPLE QUERY RAISED FROM YOUR QUESTION
# RETURNS N DOCS THAT ARE MOSTLY RELEVANT

unique_docs = retriever_from_llm.get_relevant_documents(query= question)

In [ ]:
unique_docs

In [ ]:
print(unique_docs[0].page_content)

# Context Compression

In [ ]:

from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever


In [ ]:
db_new_con = Chroma(persist_directory='./mkultra', embedding_function=HuggingFaceEmbeddings())

In [ ]:
compressor = LLMChainExtractor.from_llm(llm = ChatOpenAI())

In [ ]:
contextual_retriever = ContextualCompressionRetriever(base_compressor= compressor,
                                                    base_retriever = db_new_con.as_retriever())

In [ ]:
context_compression_results= contextual_retriever.get_relevant_documents(query= question)

In [ ]:
context_compression_results

In [ ]:
print(context_compression_results[0].metadata['summary'])

In [ ]:
print(context_compression_results[0].page_content)

# Chains

LLMChain

In [ ]:
from langchain.chains import LLMChain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

In [ ]:
human_template = "Give me a funny name for my shop that sells {product}"

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([human_template])

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
chat = ChatOpenAI()

In [ ]:
chain = LLMChain(llm=chat, prompt=chat_prompt)

In [ ]:
result = chain.run(product='Laptops')

In [ ]:
print(result)

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
chat = ChatOpenAI()

In [ ]:
template_one = " Write a simple bullet point outline for a blog post on {topic}"
first_human_prompt = ChatPromptTemplate.from_messages([template_one])
chain_one = LLMChain(llm= chat, prompt = first_human_prompt)


In [ ]:
template_two = " Write a blog post using the outline {outline}"
second_human_prompt = ChatPromptTemplate.from_messages([template_two])
chain_two = LLMChain(llm= chat, prompt = second_human_prompt)


In [ ]:
full_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose= True)

In [ ]:
result = full_chain.run('Pizza')

In [ ]:
print(result)

Sequential Chain

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
template_one = "Give a summary of the employee performance review\n{review}"
first_human_prompt = ChatPromptTemplate.from_messages([template_one])
chain_one = LLMChain(llm= chat, prompt = first_human_prompt, output_key='review_summary')

In [ ]:
template_two = "Identify the employee weaknesses from the review summary:\n{review_summary}"
second_human_prompt = ChatPromptTemplate.from_messages([template_two])
chain_two = LLMChain(llm= chat, prompt = second_human_prompt, output_key='weaknesses')

In [ ]:
template_three = "Create a personalized plan to help address and fix these weaknesses:\n{weaknesses}"
third_human_prompt = ChatPromptTemplate.from_messages([template_three])
chain_three = LLMChain(llm= chat, prompt = third_human_prompt, output_key='final_plan')

In [ ]:
full_chain = SequentialChain(chains=[chain_one, chain_two, chain_three],
                            input_variables=['review'],
                            output_variables=['review_summary', 'weaknesses', 'final_plan'],
                            verbose = True)

In [ ]:
employee_review = '''
Employee Information:
Name: Joe Schmo
Position: Software Engineer
Date of Review: July 14, 2023

Strengths:
Joe is a highly skilled software engineer with a deep understanding of programming languages, algorithms, and software development best practices. His technical expertise shines through in his ability to efficiently solve complex problems and deliver high-quality code.

One of Joe's greatest strengths is his collaborative nature. He actively engages with cross-functional teams, contributing valuable insights and seeking input from others. His open-mindedness and willingness to learn from colleagues make him a true team player.

Joe consistently demonstrates initiative and self-motivation. He takes the lead in seeking out new projects and challenges, and his proactive attitude has led to significant improvements in existing processes and systems. His dedication to self-improvement and growth is commendable.

Another notable strength is Joe's adaptability. He has shown great flexibility in handling changing project requirements and learning new technologies. This adaptability allows him to seamlessly transition between different projects and tasks, making him a valuable asset to the team.

Joe's problem-solving skills are exceptional. He approaches issues with a logical mindset and consistently finds effective solutions, often thinking outside the box. His ability to break down complex problems into manageable parts is key to his success in resolving issues efficiently.

Weaknesses:
While Joe possesses numerous strengths, there are a few areas where he could benefit from improvement. One such area is time management. Occasionally, Joe struggles with effectively managing his time, resulting in missed deadlines or the need for additional support to complete tasks on time. Developing better prioritization and time management techniques would greatly enhance his efficiency.

Another area for improvement is Joe's written communication skills. While he communicates well verbally, there have been instances where his written documentation lacked clarity, leading to confusion among team members. Focusing on enhancing his written communication abilities will help him effectively convey ideas and instructions.

Additionally, Joe tends to take on too many responsibilities and hesitates to delegate tasks to others. This can result in an excessive workload and potential burnout. Encouraging him to delegate tasks appropriately will not only alleviate his own workload but also foster a more balanced and productive team environment.
'''

In [ ]:
result = full_chain(employee_review)

In [ ]:
type(result)

In [ ]:
print(result)

In [ ]:
result.keys()

In [ ]:
print(result['weaknesses'])

In [ ]:
print(result['final_plan'])

LLMRouterChain

In [ ]:
beginner_template = '''You are a Physics teacher who is really focussed on beginners and explaining complex concepts 
in simple to understnad terms. You assume no prior knowledge. Here is your question:\n{input}'''

In [ ]:
expert_template = '''You are a Physics professor who explains physics topics to advanced Audience Members. 
You assume anyone you answer has PhD in physics. Here is your question:\n{input}'''

In [ ]:
empty_template = 'empty'

In [ ]:
prompt_infos = [{'name': 'beginner physics',
                 'description': 'Answer basic physics question',
                 'template': beginner_template},
                 
                 {'name': 'advanced physics',
                 'description': 'Answer advanced physics question',
                 'template': expert_template},
                 
                 {'name':'empty','description':'Replies to empty questions','template':empty_template},]

In [ ]:
from langchain.chains import LLMRouterChain, LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI()

LLMChain class is deprecated in newer version So, use runnable sequence

In [ ]:
destination_chains= {}

for p_info in prompt_infos:
    name = p_info['name']
    prompt_template = p_info['template']
    prompt = ChatPromptTemplate.from_template(prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    # chain = prompt | llm #runnable sequence
    destination_chains[name]= chain

In [ ]:
destination_chains

In [ ]:
default_prompt = ChatPromptTemplate.from_template('{input}')
default_chain =  LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
default_chain

In [ ]:
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [ ]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Routing Destinations

In [ ]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
print(destinations_str)

routing template

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=['input'],
    output_parser=RouterOutputParser()
)

In [ ]:
print(router_template)

Routing chain call

In [ ]:
from langchain.chains import MultiPromptChain

In [ ]:
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain= router_chain,
                        destination_chains= destination_chains,
                        default_chain= default_chain,
                        verbose= True)

In [ ]:
bg_results = chain.invoke(input="How does the magnet work?")

In [ ]:
ex_results = chain.invoke(input="How do Feynman Diagrams work?")

# Transform Chain

In [ ]:
yelp_review = '''TITLE: AN ABSOLUTE DELIGHT! A CULINARY HAVEN!

REVIEW:
OH MY GOODNESS, WHERE DO I BEGIN? THIS RESTAURANT IS ABSOLUTELY PHENOMENAL! I WENT THERE LAST NIGHT WITH MY FRIENDS, AND WE WERE BLOWN AWAY BY THE EXPERIENCE!

FIRST OF ALL, THE AMBIANCE IS OUT OF THIS WORLD! THE MOMENT YOU STEP INSIDE, YOU'RE GREETED WITH A WARM AND INVITING ATMOSPHERE. THE DECOR IS STUNNING, AND IT IMMEDIATELY SETS THE TONE FOR AN UNFORGETTABLE DINING EXPERIENCE.

NOW, LET'S TALK ABOUT THE FOOD! WOW, JUST WOW! THE MENU IS A PARADISE FOR FOOD LOVERS. EVERY DISH WE ORDERED WAS A MASTERPIECE. THE FLAVORS WERE BOLD, VIBRANT, AND EXPLODED IN OUR MOUTHS. FROM STARTERS TO DESSERTS, EVERY BITE WAS PURE BLISS!

THEIR SEAFOOD PLATTER IS A MUST-TRY! THE FRESHNESS OF THE SEAFOOD IS UNMATCHED, AND THE PRESENTATION IS SIMPLY STUNNING. I HAVE NEVER TASTED SUCH DELICIOUS AND PERFECTLY COOKED SEAFOOD IN MY LIFE. IT'S A SEAFOOD LOVER'S DREAM COME TRUE!

THE SERVICE WAS EXEMPLARY. THE STAFF WAS ATTENTIVE, FRIENDLY, AND EXTREMELY KNOWLEDGEABLE ABOUT THE MENU. THEY WENT ABOVE AND BEYOND TO ENSURE THAT WE HAD THE BEST DINING EXPERIENCE POSSIBLE.

AND LET'S NOT FORGET ABOUT THE DESSERTS! OH MY, OH MY! I HAD THEIR SIGNATURE CHOCOLATE LAVA CAKE, AND IT WAS PURE HEAVEN. THE CAKE WAS MOIST, AND WHEN I CUT INTO IT, THE WARM CHOCOLATE OOOZED OUT, CREATING AN EXPLOSION OF FLAVOR IN MY MOUTH. IT WAS LIKE A SYMPHONY OF CHOCOLATEY GOODNESS!

IN CONCLUSION, THIS RESTAURANT IS A HIDDEN GEM! IF YOU WANT TO INDULGE IN A MEMORABLE DINING EXPERIENCE, DO YOURSELF A FAVOR AND VISIT THIS PLACE. YOU WON'T REGRET IT! I CAN'T WAIT TO GO BACK AND TRY MORE OF THEIR DELECTABLE DISHES. KUDOS TO THE ENTIRE TEAM FOR CREATING SUCH A CULINARY HAVEN!

ALL I CAN SAY IS... WOOHOO!'''

In [ ]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [ ]:
def transform_fun(inputs):
    text = inputs['text']
    only_review_text = text.split('REVIEW:')[-1]
    lower_case_text = only_review_text.lower()
    return {'output': lower_case_text}

In [ ]:
transform_chain = TransformChain(input_variables=['text'],
                                output_variables=['output'],
                                transform= transform_fun,
                                verbose= True)

In [ ]:
template = "Give me an one line summary of this review:\n{review}"
summary_chain = LLMChain(llm= ChatOpenAI(), prompt = ChatPromptTemplate.from_template(template), output_key='review_sumamry')

In [ ]:
full_chain = SimpleSequentialChain(chains=[transform_chain, summary_chain],
                                    verbose=True)

In [ ]:
result = full_chain(yelp_review)

In [ ]:
result

# Openai Function calling

In [ ]:
chat = ChatOpenAI()

In [ ]:
json_schema = {"title": "Scientist",
               "description": "Information about a famous scientist",
               "type": "object",
               "properties":{
                   "first_name":{'title':'First Name',
                                 'description': "First name of scientist",
                                 "type": "string"},
                   "last_name":{'title':'Last Name',
                                 'description': "Last name of scientist",
                                 "type": "string"},
               },
                "required": ['first_name','last_name']
              }

In [ ]:

from langchain.chains.openai_functions import (
    create_structured_output_chain,
)

In [ ]:
prompt = ChatPromptTemplate.from_template('Name a Famous {Country} Actor')
chain = create_structured_output_chain(json_schema, llm= chat, prompt=prompt)

In [ ]:
result = chain.invoke('American')

In [ ]:
result

# Additional Chains

In [ ]:
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

In [ ]:
from langchain.schema import HumanMessage

In [ ]:
db_con = Chroma(persist_directory='./mkultra', embedding_function=HuggingFaceEmbeddings())

In [ ]:
question= "{context}"

In [ ]:
docs = db_con.similarity_search(question.format(context="When was this declassified?"))

In [ ]:
docs

In [ ]:
question= "{input_question}: {context}"

In [ ]:

human_prompt = HumanMessagePromptTemplate.from_template(question)
prompt = ChatPromptTemplate.from_messages([human_prompt])
stuff_chain = create_stuff_documents_chain(llm=ChatOpenAI(), prompt=prompt)

In [ ]:
prompt.input_variables

In [ ]:
stuff_chain.invoke({'context': docs, 'input_question': "When was this declassified?"})

# Memory

In [ ]:
from langchain.memory import ChatMessageHistory

In [ ]:
history = ChatMessageHistory()

In [ ]:
history.add_user_message("Hi Nice to meet you")

In [ ]:
history.add_ai_message("Hello user, Nice to meet you!")

In [ ]:
history.messages

ConversationBufferMemory

In [ ]:
from langchain.memory import ConversationBufferMemory

In [ ]:
memory= ConversationBufferMemory()

In [ ]:
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI

In [ ]:
conversation = ConversationChain(llm=ChatOpenAI(), memory=memory, verbose=True)

In [ ]:
conversation.predict(input="Hello there")

In [ ]:
conversation.invoke(input="Tell me an interesting fact")

In [ ]:
conversation.predict(input="Oh that's a brand new information for me.")

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})

In [ ]:
conversation.memory

In [ ]:
import pickle

In [ ]:
pickeled_str = pickle.dumps(conversation.memory)

In [ ]:
with open('convo_memory.pkl', 'wb') as f:
    f.write(pickeled_str)

In [ ]:
old_memory_load = open('convo_memory.pkl', 'rb').read()
old_memory_load=pickle.loads(old_memory_load)

In [ ]:
reload_convo = ConversationChain(llm=ChatOpenAI(), memory=old_memory_load, verbose=True)

In [ ]:
reload_convo.memory

In [ ]:
print(reload_convo.memory.buffer)

In [ ]:
reload_convo.predict(input='Thanks for helping me')

In [ ]:
print(reload_convo.memory.buffer)

In [ ]:
old_memory_load.load_memory_variables({})

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
win_memory = ConversationBufferWindowMemory(k=1)

In [ ]:
convo = ConversationChain(llm=ChatOpenAI(), memory=win_memory, verbose=True)

In [ ]:
convo.predict(input="Hello I am an ML engineer")

In [ ]:
convo.predict(input="You are doing an amzing job as an assistant")

In [ ]:
print(memory.buffer)

In [ ]:
print(convo.memory.buffer)

In [ ]:
win_memory.load_memory_variables({}) #k=1

In [ ]:
from langchain.memory import ConversationSummaryMemory

In [ ]:
memory = ConversationSummaryMemory(llm=ChatOpenAI(), max_token_limit=100)

In [ ]:
sum_convo = ConversationChain(llm=ChatOpenAI(), memory=memory, verbose=True)

In [ ]:
sum_convo.predict(input="Tell me the distance of moon from the earth?")

In [ ]:
sum_convo.predict(input="Is Mars is closer to earth than moon?")

In [ ]:
sum_convo.memory.buffer

In [ ]:
memory.load_memory_variables({})

# Agents

In [ ]:
pip install -U langchain-community

In [ ]:
from langchain.agents import load_tools, initialize_agent, AgentType

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
from langchain.llms import OpenAI

In [ ]:
! pip install numexpr

In [ ]:
llm = ChatOpenAI(temperature=0)
tools = load_tools(['llm-math'], llm=llm)

In [ ]:
type(tools)

In [ ]:
dir(AgentType)

In [ ]:
agent = initialize_agent(tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.invoke(input="What is 1342 times of 6827?")

In [ ]:
1342 *6827

Custom Tools

In [ ]:
agent.run("What is the current time now?")

In [ ]:
from datetime import datetime
from langchain.agents import tool

In [ ]:
@tool
def get_curr_time(text: str) -> str:
    '''Returns the current time. use this only for any questions on current time.
    Input is an empty string '' and teh curren time is returned in string format, other time related should
    use other tools'''
    return str(datetime.now())

In [ ]:
agent = initialize_agent(tools=tools+[get_curr_time], llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("What is the current time?")

Conversation Agent

In [ ]:
from langchain.memory import ConversationBufferMemory

In [ ]:
memory = ConversationBufferMemory(memory_key='chat_history')

In [ ]:
llm = ChatOpenAI()

In [ ]:
tools = load_tools(['llm-math'], llm=llm)

In [ ]:
agent = initialize_agent(tools=tools, llm=llm, memory=memory, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("Give me two Indian recipes")

In [ ]:
agent.run("Which of the two is less spicy?")

In [ ]:
agent.run("Give me the grocery shopping list for that dish?")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

In [ ]:
# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

# 2. Create model
model = ChatOpenAI()

# 3. Create parser
parser = StrOutputParser()

# 4. Create chain
chain = prompt_template | model | parser

In [ ]:
chain.invoke(input={"text": "Hello world", "language": "French"})